In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv

prep571.csv  prep573.csv  prep712.csv  prep714.csv  prep716.csv  prep718.csv
prep572.csv  prep711.csv  prep713.csv  prep715.csv  prep717.csv


In [ ]:
%cd /content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv

/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv


In [ ]:
import tensorflow as tf
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import math
import scipy.stats as stats 
from numpy import array
from pandas import read_csv
from math import sqrt
from numpy import split
from plotly.subplots import make_subplots
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from datetime import datetime
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

## Visualisasi Data Mentah

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv/prep571.csv')
data = data.sort_values('time')

data = data.reset_index(drop=True, inplace=False)
# print(data)

# kolom_interpolasi = ['swh']
data['tp'] = data['tp'].interpolate(method='linear', limit_direction='both')


# data['swh'] = imputer.fit_transform(data['swh'].values.reshape(-1, 1))


# Assuming you have a DataFrame named 'data' containing SWH data with a column named 'swh'
# Check for NaN values in the 'swh' column
nan_values = data['tp'].isna().any()
nan_percentage = (data['tp'].isna().sum() / len(data['tp'])) * 100

# Print the result
print("Percentage of NaN values in the SWH data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the SWH data.")
else:
    print("No NaN values found in the SWH data.")
print(data)
# daerah = data

# data = preparedata(daerah)

Percentage of NaN values in the SWH data: 0.00%
No NaN values found in the SWH data.
       longitude  latitude                 time        tp  expver
0          100.5       5.0  2022-01-01 00:00:00  0.000039     NaN
1          100.5       5.0  2022-01-01 01:00:00  0.000053     NaN
2          100.5       5.0  2022-01-01 02:00:00  0.000027     NaN
3          100.5       5.0  2022-01-01 03:00:00  0.000029     NaN
4          100.5       5.0  2022-01-01 04:00:00  0.000078     NaN
...          ...       ...                  ...       ...     ...
14515      100.5       5.0  2023-04-30 21:00:00  0.000003     5.0
14516      100.5       5.0  2023-04-30 22:00:00  0.000044     1.0
14517      100.5       5.0  2023-04-30 22:00:00  0.000086     5.0
14518      100.5       5.0  2023-04-30 23:00:00  0.000044     1.0
14519      100.5       5.0  2023-04-30 23:00:00  0.000003     5.0

[14520 rows x 5 columns]


In [ ]:

# Menampilkan grafik
plt.show()


fig = go.Figure()
fig.add_trace(go.Scatter(x=data['time'], y=data['tp'],
                         mode='lines', name='Data Presipitasi'))

# Edit the layout
fig.update_layout(xaxis_title='<b>Date</b>', yaxis_title='<b><i>Hs</i> [m]</b>')

fig.show()

## Simple Model

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.metrics import Accuracy

# Load and preprocess the data
swh = data["tp"].values.reshape(-1, 1)  # Reshape to a column vector
scaler = MinMaxScaler()
swh_scaled = scaler.fit_transform(swh)

# Split the data into training and testing sets
train_size = int(len(swh_scaled) * 0.8)
train_data = swh_scaled[:train_size]
test_data = swh_scaled[train_size:]

# Define the function to create the input sequences
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        target = data[i+seq_length]
        X.append(seq)
        y.append(target)
    return np.array(X), np.array(y)

# Set the sequence length and create the input sequences
seq_length = 10  # Adjust as needed
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

# Define the LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(seq_length, 1)),
    tf.keras.layers.Dense(1)
])

# Create an instance of the Accuracy metric
# accuracy_metric = Accuracy()

# Compile the model and add the accuracy metric
model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])


# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=2)

# Evaluate the model
train_loss = model.evaluate(X_train, y_train)
test_loss = model.evaluate(X_test, y_test)

# Make predictions
predictions = model.predict(X_test)

# Inverse scale the predictions and actual values
predictions = scaler.inverse_transform(predictions)
actual_values = scaler.inverse_transform(y_test)



Epoch 1/10
363/363 - 7s - loss: 0.3039 - accuracy: 0.1412 - 7s/epoch - 19ms/step
Epoch 2/10
363/363 - 3s - loss: 0.3039 - accuracy: 0.1412 - 3s/epoch - 8ms/step
Epoch 3/10
363/363 - 3s - loss: 0.3039 - accuracy: 0.1412 - 3s/epoch - 9ms/step
Epoch 4/10
363/363 - 3s - loss: 0.3039 - accuracy: 0.1412 - 3s/epoch - 9ms/step
Epoch 5/10
363/363 - 4s - loss: 0.3039 - accuracy: 0.1412 - 4s/epoch - 12ms/step
Epoch 6/10
363/363 - 3s - loss: 0.3039 - accuracy: 0.1412 - 3s/epoch - 8ms/step
Epoch 7/10
363/363 - 3s - loss: 0.3039 - accuracy: 0.1412 - 3s/epoch - 9ms/step
Epoch 8/10
363/363 - 2s - loss: 0.3039 - accuracy: 0.1412 - 2s/epoch - 7ms/step
Epoch 9/10
363/363 - 2s - loss: 0.3039 - accuracy: 0.1412 - 2s/epoch - 7ms/step
Epoch 10/10
363/363 - 5s - loss: 0.3039 - accuracy: 0.1412 - 5s/epoch - 13ms/step
91/91 [==============================] - 1s 3ms/step


In [ ]:
# Calculate RMSE
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('rmse:',rmse)
# print("Accuracy: {:.2f}%".format(accuracy * 100))
# Print RMSE
# print("RMSE: {:.2f}".format(rmse))
# # Print the predictions and actual values
# for i in range(len(predictions)):
#     print("Predicted:", predictions[i][0], "Actual:", actual_values[i][0])

91/91 [==============================] - 0s 3ms/step
rmse: 0.04351513118154341


In [ ]:
import plotly.graph_objects as go

# Create the figure
fig = go.Figure()

# Add the predicted values trace
fig.add_trace(go.Scatter(
    x=list(range(len(y_test))),
    y=y_pred.flatten(),
    mode='lines',
    name='Predicted Values'
))

# Add the actual values trace
fig.add_trace(go.Scatter(
    x=list(range(len(y_test))),
    y=y_test.flatten(),
    mode='lines',
    name='Actual Values'
))

# Set the layout
fig.update_layout(
    title='Predicted Values vs Actual Values',
    xaxis_title='Time',
    yaxis_title='Value',
)

# Show the figure
fig.show()


# Fungsi


## splitting data

In [ ]:
# convert data into x and y 
def change_data(train, n_input, n_out=3):
	data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
	X, y = list(), list()
	in_start = 0
	for _ in range(n_input, len(data)):
		in_end = in_start + n_input
		out_end = in_end + n_out
		if out_end <= len(data):
			X.append(data[in_start:in_end, :])
			y.append(data[in_end:out_end, 0])
		in_start += 1
	return array(X), array(y)

In [ ]:
def splitting_data(data):
  # end_data = list(data['time']).index(datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S' ))
  data= data.loc[:,['tp']]
  data= data.reset_index(drop=True)
 
  # scaler_label = StandardScaler().fit(data['swh'].values.reshape(-1, 1))
  # scaler_label = scaler_label.transform(data['swh'].values.reshape(-1, 1))

  scaler_label = StandardScaler().fit(data['tp'].values.reshape(-1, 1))
  scaler_label = scaler_label.transform(data['tp'].values.reshape(-1, 1))

  print(scaler_label)
  data_new = pd.DataFrame(scaler_label,index = data.index, columns=data.columns)
  train = data_new[:round(len(data_new)*0.8)]
  val = data_new[round(len(data_new)*0.8) : round(len(data_new)*0.9) ]
  test = data_new[round(len(data_new)*0.9):]

  i = 0
  while len(train) % 3 != 0:
    train = train[:round(len(train)*0.8) + i]
    i += 1
  j = 0
  while len(test) % 3 != 0:
    test = test[round(len(test)*0.9) - j:]
    j += 1
  
  val = val[round(len(val)*0.8)+i : round(len(val)*0.9)-j]
  while len(val) % 3 != 0:
    i -= 1
    val = val[round(len(val)*0.8)+i : round(len(val)*0.9)-j]


  # resturvture into windows of 3 hours data
  train = array(split(train, round(len(train)/3)))
  val = array(split(val, round(len(val)/3)))
  test = array(split(test, round(len(test)/3)))

  n_input,n_out = 9,3

  n_steps_in, n_steps_out = 9, 3
  X_train, y_train =  change_data(train,n_steps_in, n_steps_out)
  X_val, y_val =  change_data(val,n_steps_in, n_steps_out)
  
  return X_train, y_train, X_val, y_val, train, val, test ,j

## model 

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') is not None and logs.get('accuracy') > 0.99:
            print("\nReached 99% accuracy so cancelling training!")
            self.model.stop_training = True    

def define_model():
  tf.keras.backend.clear_session()
  n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
    # Build the LSTM model
  model = Sequential()
  model.add(LSTM(units=64, return_sequences=True, input_shape=(n_timesteps, n_features)))
  model.add(Dropout(0.2))
  model.add(LSTM(units=64, return_sequences=True))
  model.add(Dropout(0.2))
  model.add(LSTM(units=64))
  model.add(Dropout(0.2))
  model.add(Dense(units=3))
  # cp1 = ModelCheckpoint('model1/', save_best_only=True)
  model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.001), 
                 metrics=['accuracy'])
    
  return model

def fit_model(X_train, y_train, X_val, y_val):
  # Instantiate the callback
  callbacks = myCallback()
  
  # fitting model 
  history = model.fit(X_train, y_train, validation_data=(X_val, y_val), 
                       epochs=10, batch_size=32,callbacks=callbacks,verbose=2)

  # EXERCISE: Save the trained model as a Keras HDF5 file. 
  # saved_model_path = '/content/drive/MyDrive/overfishnt/data/fishing_spot_heatmap/o2/model/'+daerah+'.h5'
  # YOUR CODE HERE
  
  # model1.save(saved_model_path)
  return history

In [ ]:
def forecast(data, daerah):
    scaler_label = StandardScaler().fit(data['tp'].values.reshape(-1, 1))
    data_new = data

    history = [x for x in val]
    predictions = []
    n_input = 9
    for k in range(len(test)):
        datax = np.array(history)
        datax = datax.reshape((datax.shape[0] * datax.shape[1], datax.shape[2]))
        input_x = datax[-n_input:, :]
        input_x = input_x.reshape((1, len(input_x), 1))
        # predict the week
        yhat = model.predict(input_x, verbose=0)
        yhat_sequence = yhat[0]
        # store the predictions
        predictions.append(yhat_sequence)
        # get real observation and add to history for predicting the next week
        history.append(test[k, :])

    predictions = np.array(predictions)
    predictions = scaler_label.inverse_transform(predictions)

    observasi = test
    observasi = test[:, :, 0]
    observasi = scaler_label.inverse_transform(observasi)

    predictions = predictions.reshape((predictions.shape[0] * predictions.shape[1], 1))
    observasi = observasi.reshape((observasi.shape[0] * observasi.shape[1], 1))

    test_results = pd.DataFrame(data={'Predictions': predictions.flatten(), 'Actuals': observasi.flatten()})
    test_results['Date'] = data_new['time'].iloc[round(len(data_new) * 0.9) - j:].values

    return test_results

def forecast_viz():
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=test_results['Date'], y=test_results['Predictions'],
                             mode='lines',
                             name='Fitting Data Testing'))
    fig.add_trace(go.Scatter(x=test_results['Date'], y=test_results['Actuals'],
                             mode='lines',
                             name='Observation'))

    # Edit the layout
    fig.update_layout(xaxis_title='<b>Date</b>', yaxis_title='<b><i>Hs</i> [m]</b>')

    return fig.show()


## Evaluating Model

In [ ]:
def evaluate():
  MSE = mean_squared_error(test_results['Actuals'],test_results['Predictions'])
  RMSE = math.sqrt(MSE)
  corr, _ = pearsonr(test_results['Actuals'],test_results['Predictions'])

  gap_sum = 0
  for j in range(len(test_results['Actuals'])):
    gap = test_results['Predictions'][j] - test_results['Actuals'][j]
    gap_sum += gap
    
  bias = gap_sum/len(test_results['Actuals'])
  _, pvalue = stats.ttest_ind(test_results['Actuals'],test_results['Predictions'], equal_var=True)

  value = [round(RMSE,5),round(corr,5), round(bias,5), round(pvalue,5)]
  eval = pd.DataFrame(value, index =['rmse', 'corr', 'bias','pvalue'],
                                              columns =['values'])
  return eval

# WPP571

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv/prep571.csv')
data = data.sort_values('time')

data = data.reset_index(drop=True, inplace=False)
print(data)

       longitude  latitude                 time        tp  expver
0          100.5       5.0  2022-01-01 00:00:00  0.000039     NaN
1          100.5       5.0  2022-01-01 01:00:00  0.000053     NaN
2          100.5       5.0  2022-01-01 02:00:00  0.000027     NaN
3          100.5       5.0  2022-01-01 03:00:00  0.000029     NaN
4          100.5       5.0  2022-01-01 04:00:00  0.000078     NaN
...          ...       ...                  ...       ...     ...
14515      100.5       5.0  2023-04-30 21:00:00  0.000003     5.0
14516      100.5       5.0  2023-04-30 22:00:00       NaN     1.0
14517      100.5       5.0  2023-04-30 22:00:00  0.000086     5.0
14518      100.5       5.0  2023-04-30 23:00:00       NaN     1.0
14519      100.5       5.0  2023-04-30 23:00:00  0.000003     5.0

[14520 rows x 5 columns]


### Interpolate the missing values

In [ ]:
data['tp'] = data['tp'].interpolate(method='quadratic', limit_direction='both')
data['tp'] = data['tp'].clip(lower=0)
#check the nan  values
nan_values = data['tp'].isna().any()
nan_percentage = (data['tp'].isna().sum() / len(data['tp'])) * 100
print((data))

       longitude  latitude                 time        tp  expver
0          100.5       5.0  2022-01-01 00:00:00  0.000039     NaN
1          100.5       5.0  2022-01-01 01:00:00  0.000053     NaN
2          100.5       5.0  2022-01-01 02:00:00  0.000027     NaN
3          100.5       5.0  2022-01-01 03:00:00  0.000029     NaN
4          100.5       5.0  2022-01-01 04:00:00  0.000078     NaN
...          ...       ...                  ...       ...     ...
14515      100.5       5.0  2023-04-30 21:00:00  0.000003     5.0
14516      100.5       5.0  2023-04-30 22:00:00  0.000049     1.0
14517      100.5       5.0  2023-04-30 22:00:00  0.000086     5.0
14518      100.5       5.0  2023-04-30 23:00:00  0.000071     1.0
14519      100.5       5.0  2023-04-30 23:00:00  0.000003     5.0

[14520 rows x 5 columns]


### Check if there are any nan values

In [ ]:
# Print the result
print("Percentage of NaN values in the Precipitation data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the Precipitation data.")
else:
    print("No NaN values found in the Precipitation data.")
print(data)
# daerah = data

Percentage of NaN values in the Precipitation data: 0.00%
No NaN values found in the Precipitation data.
       longitude  latitude                 time        tp  expver
0          100.5       5.0  2022-01-01 00:00:00  0.000039     NaN
1          100.5       5.0  2022-01-01 01:00:00  0.000053     NaN
2          100.5       5.0  2022-01-01 02:00:00  0.000027     NaN
3          100.5       5.0  2022-01-01 03:00:00  0.000029     NaN
4          100.5       5.0  2022-01-01 04:00:00  0.000078     NaN
...          ...       ...                  ...       ...     ...
14515      100.5       5.0  2023-04-30 21:00:00  0.000003     5.0
14516      100.5       5.0  2023-04-30 22:00:00  0.000049     1.0
14517      100.5       5.0  2023-04-30 22:00:00  0.000086     5.0
14518      100.5       5.0  2023-04-30 23:00:00  0.000071     1.0
14519      100.5       5.0  2023-04-30 23:00:00  0.000003     5.0

[14520 rows x 5 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[-0.41192108]
 [-0.39169925]
 [-0.43113211]
 ...
 [-0.34063498]
 [-0.36418749]
 [-0.46777021]]


((11605, 9, 1), (11605, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3872, 3, 1), (5, 3, 1), (484, 3, 1))

In [ ]:
# fitting model to data 
model = define_model()
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
363/363 - 18s - loss: 0.8255 - accuracy: 0.3698 - val_loss: 0.1285 - val_accuracy: 0.7500 - 18s/epoch - 51ms/step
Epoch 2/10
363/363 - 9s - loss: 0.7057 - accuracy: 0.3928 - val_loss: 0.1331 - val_accuracy: 0.7500 - 9s/epoch - 24ms/step
Epoch 3/10
363/363 - 14s - loss: 0.6835 - accuracy: 0.3941 - val_loss: 0.1301 - val_accuracy: 1.0000 - 14s/epoch - 39ms/step
Epoch 4/10
363/363 - 16s - loss: 0.6776 - accuracy: 0.4047 - val_loss: 0.1396 - val_accuracy: 0.7500 - 16s/epoch - 44ms/step
Epoch 5/10
363/363 - 11s - loss: 0.6742 - accuracy: 0.4051 - val_loss: 0.1413 - val_accuracy: 1.0000 - 11s/epoch - 31ms/step
Epoch 6/10
363/363 - 10s - loss: 0.6678 - accuracy: 0.4050 - val_loss: 0.1342 - val_accuracy: 1.0000 - 10s/epoch - 29ms/step
Epoch 7/10
363/363 - 8s - loss: 0.6522 - accuracy: 0.4083 - val_loss: 0.1409 - val_accuracy: 1.0000 - 8s/epoch - 22ms/step
Epoch 8/10
363/363 - 10s - loss: 0.6536 - accuracy: 0.4075 - val_loss: 0.1490 - val_accuracy: 1.0000 - 10s/epoch - 27ms/step
Epoc

In [ ]:
test_results = forecast(data,'WPP571')      
test_results

,Predictions,Actuals,Date
0,0.000741,0.000195,2023-03-31 18:00:00
1,0.000617,0.000052,2023-03-31 18:00:00
2,0.000538,0.000013,2023-03-31 19:00:00
3,0.000058,0.000003,2023-03-31 19:00:00
4,0.000139,0.000004,2023-03-31 20:00:00
...,...,...,...
1447,0.000108,0.000003,2023-04-30 21:00:00
1448,0.000146,0.000049,2023-04-30 22:00:00
1449,0.000118,0.000086,2023-04-30 22:00:00
1450,0.000144,0.000071,2023-04-30 23:00:00


In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

,values
rmse,0.00045
corr,0.70717
bias,0.00004
pvalue,0.05633


In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv'+'presipitasiwpp571'+'.h5'
model.save(saved_model_path)

# WPP572

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv/prep572.csv')
data = data.sort_values('time')

data = data.reset_index(drop=True, inplace=False)
print(data)

       longitude  latitude                 time        tp  expver
0          100.0      -4.0  2022-01-01 00:00:00  0.000079     NaN
1          100.0      -4.0  2022-01-01 01:00:00  0.000096     NaN
2          100.0      -4.0  2022-01-01 02:00:00  0.000617     NaN
3          100.0      -4.0  2022-01-01 03:00:00  0.001365     NaN
4          100.0      -4.0  2022-01-01 04:00:00  0.000763     NaN
...          ...       ...                  ...       ...     ...
14515      100.0      -4.0  2023-04-30 21:00:00  0.000003     5.0
14516      100.0      -4.0  2023-04-30 22:00:00       NaN     1.0
14517      100.0      -4.0  2023-04-30 22:00:00  0.000002     5.0
14518      100.0      -4.0  2023-04-30 23:00:00       NaN     1.0
14519      100.0      -4.0  2023-04-30 23:00:00  0.000006     5.0

[14520 rows x 5 columns]


### Interpolate the missing values

In [ ]:
data['tp'] = data['tp'].interpolate(method='quadratic', limit_direction='both')
data['tp'] = data['tp'].clip(lower=0)
#check the nan  values
nan_values = data['tp'].isna().any()
nan_percentage = (data['tp'].isna().sum() / len(data['tp'])) * 100
print((data))

       longitude  latitude                 time        tp  expver
0          100.0      -4.0  2022-01-01 00:00:00  0.000079     NaN
1          100.0      -4.0  2022-01-01 01:00:00  0.000096     NaN
2          100.0      -4.0  2022-01-01 02:00:00  0.000617     NaN
3          100.0      -4.0  2022-01-01 03:00:00  0.001365     NaN
4          100.0      -4.0  2022-01-01 04:00:00  0.000763     NaN
...          ...       ...                  ...       ...     ...
14515      100.0      -4.0  2023-04-30 21:00:00  0.000003     5.0
14516      100.0      -4.0  2023-04-30 22:00:00  0.000002     1.0
14517      100.0      -4.0  2023-04-30 22:00:00  0.000002     5.0
14518      100.0      -4.0  2023-04-30 23:00:00  0.000003     1.0
14519      100.0      -4.0  2023-04-30 23:00:00  0.000006     5.0

[14520 rows x 5 columns]


### Check if there are any nan values

In [ ]:
# Print the result
print("Percentage of NaN values in the Precipitation data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the Precipitation data.")
else:
    print("No NaN values found in the Precipitation data.")
print(data)
# daerah = data

Percentage of NaN values in the Precipitation data: 0.00%
No NaN values found in the Precipitation data.
       longitude  latitude                 time        tp  expver
0          100.0      -4.0  2022-01-01 00:00:00  0.000079     NaN
1          100.0      -4.0  2022-01-01 01:00:00  0.000096     NaN
2          100.0      -4.0  2022-01-01 02:00:00  0.000617     NaN
3          100.0      -4.0  2022-01-01 03:00:00  0.001365     NaN
4          100.0      -4.0  2022-01-01 04:00:00  0.000763     NaN
...          ...       ...                  ...       ...     ...
14515      100.0      -4.0  2023-04-30 21:00:00  0.000003     5.0
14516      100.0      -4.0  2023-04-30 22:00:00  0.000002     1.0
14517      100.0      -4.0  2023-04-30 22:00:00  0.000002     5.0
14518      100.0      -4.0  2023-04-30 23:00:00  0.000003     1.0
14519      100.0      -4.0  2023-04-30 23:00:00  0.000006     5.0

[14520 rows x 5 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[-0.37038575]
 [-0.34950661]
 [ 0.28653604]
 ...
 [-0.46446014]
 [-0.46290982]
 [-0.45987036]]


((11605, 9, 1), (11605, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3872, 3, 1), (5, 3, 1), (484, 3, 1))

In [ ]:
# fitting model to data 
model = define_model()
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
363/363 - 17s - loss: 0.7678 - accuracy: 0.3497 - val_loss: 0.0040 - val_accuracy: 0.5000 - 17s/epoch - 46ms/step
Epoch 2/10
363/363 - 8s - loss: 0.5897 - accuracy: 0.3919 - val_loss: 0.0024 - val_accuracy: 0.5000 - 8s/epoch - 21ms/step
Epoch 3/10
363/363 - 12s - loss: 0.5559 - accuracy: 0.4031 - val_loss: 0.0174 - val_accuracy: 0.5000 - 12s/epoch - 34ms/step
Epoch 4/10
363/363 - 16s - loss: 0.5470 - accuracy: 0.4065 - val_loss: 0.0019 - val_accuracy: 0.5000 - 16s/epoch - 45ms/step
Epoch 5/10
363/363 - 10s - loss: 0.5449 - accuracy: 0.4165 - val_loss: 0.0033 - val_accuracy: 0.5000 - 10s/epoch - 28ms/step
Epoch 6/10
363/363 - 9s - loss: 0.5247 - accuracy: 0.4101 - val_loss: 0.0026 - val_accuracy: 0.5000 - 9s/epoch - 26ms/step
Epoch 7/10
363/363 - 8s - loss: 0.5317 - accuracy: 0.4202 - val_loss: 0.0075 - val_accuracy: 0.5000 - 8s/epoch - 21ms/step
Epoch 8/10
363/363 - 8s - loss: 0.5262 - accuracy: 0.4102 - val_loss: 0.0046 - val_accuracy: 0.5000 - 8s/epoch - 23ms/step
Epoch 9/

In [ ]:
test_results = forecast(data,'WPP572')
test_results

,Predictions,Actuals,Date
0,0.000075,0.000035,2023-03-31 18:00:00
1,0.000081,0.000020,2023-03-31 18:00:00
2,0.000112,0.000003,2023-03-31 19:00:00
3,0.000054,0.000003,2023-03-31 19:00:00
4,0.000067,0.000017,2023-03-31 20:00:00
...,...,...,...
1447,0.000062,0.000003,2023-04-30 21:00:00
1448,0.000093,0.000002,2023-04-30 22:00:00
1449,0.000051,0.000002,2023-04-30 22:00:00
1450,0.000059,0.000003,2023-04-30 23:00:00


In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

,values
rmse,0.00025
corr,0.78340
bias,0.00005
pvalue,0.00004


In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv'+'presipitasiwpp572'+'.h5'
model.save(saved_model_path)

# WPP573

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv/prep573.csv')
data = data.sort_values('time')

data = data.reset_index(drop=True, inplace=False)
print(data)

       longitude  latitude                 time        tp  expver
0          112.5      -9.5  2022-01-01 00:00:00  0.000015     NaN
1          112.5      -9.5  2022-01-01 01:00:00  0.000034     NaN
2          112.5      -9.5  2022-01-01 02:00:00  0.000036     NaN
3          112.5      -9.5  2022-01-01 03:00:00  0.000016     NaN
4          112.5      -9.5  2022-01-01 04:00:00  0.000026     NaN
...          ...       ...                  ...       ...     ...
14515      112.5      -9.5  2023-04-30 21:00:00  0.000083     5.0
14516      112.5      -9.5  2023-04-30 22:00:00       NaN     1.0
14517      112.5      -9.5  2023-04-30 22:00:00  0.000058     5.0
14518      112.5      -9.5  2023-04-30 23:00:00       NaN     1.0
14519      112.5      -9.5  2023-04-30 23:00:00  0.000095     5.0

[14520 rows x 5 columns]


### Interpolate the missing values

In [ ]:
data['tp'] = data['tp'].interpolate(method='quadratic', limit_direction='both')
data['tp'] = data['tp'].clip(lower=0)
#check the nan  values
nan_values = data['tp'].isna().any()
nan_percentage = (data['tp'].isna().sum() / len(data['tp'])) * 100
print((data))

       longitude  latitude                 time        tp  expver
0          112.5      -9.5  2022-01-01 00:00:00  0.000015     NaN
1          112.5      -9.5  2022-01-01 01:00:00  0.000034     NaN
2          112.5      -9.5  2022-01-01 02:00:00  0.000036     NaN
3          112.5      -9.5  2022-01-01 03:00:00  0.000016     NaN
4          112.5      -9.5  2022-01-01 04:00:00  0.000026     NaN
...          ...       ...                  ...       ...     ...
14515      112.5      -9.5  2023-04-30 21:00:00  0.000083     5.0
14516      112.5      -9.5  2023-04-30 22:00:00  0.000066     1.0
14517      112.5      -9.5  2023-04-30 22:00:00  0.000058     5.0
14518      112.5      -9.5  2023-04-30 23:00:00  0.000068     1.0
14519      112.5      -9.5  2023-04-30 23:00:00  0.000095     5.0

[14520 rows x 5 columns]


### Check if there are any nan values

In [ ]:
# Print the result
print("Percentage of NaN values in the Precipitation data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the Precipitation data.")
else:
    print("No NaN values found in the Precipitation data.")
print(data)
# daerah = data

Percentage of NaN values in the Precipitation data: 0.00%
No NaN values found in the Precipitation data.
       longitude  latitude                 time        tp  expver
0          112.5      -9.5  2022-01-01 00:00:00  0.000015     NaN
1          112.5      -9.5  2022-01-01 01:00:00  0.000034     NaN
2          112.5      -9.5  2022-01-01 02:00:00  0.000036     NaN
3          112.5      -9.5  2022-01-01 03:00:00  0.000016     NaN
4          112.5      -9.5  2022-01-01 04:00:00  0.000026     NaN
...          ...       ...                  ...       ...     ...
14515      112.5      -9.5  2023-04-30 21:00:00  0.000083     5.0
14516      112.5      -9.5  2023-04-30 22:00:00  0.000066     1.0
14517      112.5      -9.5  2023-04-30 22:00:00  0.000058     5.0
14518      112.5      -9.5  2023-04-30 23:00:00  0.000068     1.0
14519      112.5      -9.5  2023-04-30 23:00:00  0.000095     5.0

[14520 rows x 5 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[-0.37631573]
 [-0.34526146]
 [-0.34193522]
 ...
 [-0.30357203]
 [-0.28738547]
 [-0.24229583]]


((11605, 9, 1), (11605, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3872, 3, 1), (5, 3, 1), (484, 3, 1))

In [ ]:
# fitting model to data 
model = define_model()
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
363/363 - 17s - loss: 0.6572 - accuracy: 0.3437 - val_loss: 0.0136 - val_accuracy: 0.0000e+00 - 17s/epoch - 48ms/step
Epoch 2/10
363/363 - 7s - loss: 0.4781 - accuracy: 0.3503 - val_loss: 0.0021 - val_accuracy: 0.0000e+00 - 7s/epoch - 20ms/step
Epoch 3/10
363/363 - 9s - loss: 0.4633 - accuracy: 0.3598 - val_loss: 0.0028 - val_accuracy: 0.0000e+00 - 9s/epoch - 24ms/step
Epoch 4/10
363/363 - 7s - loss: 0.4467 - accuracy: 0.3690 - val_loss: 0.0020 - val_accuracy: 0.0000e+00 - 7s/epoch - 20ms/step
Epoch 5/10
363/363 - 8s - loss: 0.4435 - accuracy: 0.3795 - val_loss: 0.0018 - val_accuracy: 0.0000e+00 - 8s/epoch - 23ms/step
Epoch 6/10
363/363 - 7s - loss: 0.4384 - accuracy: 0.3721 - val_loss: 0.0077 - val_accuracy: 0.0000e+00 - 7s/epoch - 18ms/step
Epoch 7/10
363/363 - 9s - loss: 0.4328 - accuracy: 0.3690 - val_loss: 0.0019 - val_accuracy: 0.0000e+00 - 9s/epoch - 24ms/step
Epoch 8/10
363/363 - 7s - loss: 0.4287 - accuracy: 0.3708 - val_loss: 0.0025 - val_accuracy: 0.0000e+00 - 7s/

In [ ]:
test_results = forecast(data,'WPP573')
test_results

,Predictions,Actuals,Date
0,0.000016,0.001384,2023-03-31 18:00:00
1,0.000029,0.000784,2023-03-31 18:00:00
2,0.000056,0.000267,2023-03-31 19:00:00
3,0.000321,0.000061,2023-03-31 19:00:00
4,0.000364,0.000144,2023-03-31 20:00:00
...,...,...,...
1447,0.000169,0.000083,2023-04-30 21:00:00
1448,0.000205,0.000066,2023-04-30 22:00:00
1449,0.000076,0.000058,2023-04-30 22:00:00
1450,0.000098,0.000068,2023-04-30 23:00:00


In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

,values
rmse,0.00020
corr,0.82922
bias,0.00001
pvalue,0.28411


In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv'+'presipitasiwpp573'+'.h5'
model.save(saved_model_path)

# WPP711

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv/prep711.csv')
data = data.sort_values('time')

data = data.reset_index(drop=True, inplace=False)
print(data)

       longitude  latitude                 time        tp  expver
0          108.5      -0.5  2022-01-01 00:00:00  0.000303     NaN
1          108.5      -0.5  2022-01-01 01:00:00  0.000587     NaN
2          108.5      -0.5  2022-01-01 02:00:00  0.000704     NaN
3          108.5      -0.5  2022-01-01 03:00:00  0.000229     NaN
4          108.5      -0.5  2022-01-01 04:00:00  0.000057     NaN
...          ...       ...                  ...       ...     ...
14515      108.5      -0.5  2023-04-30 21:00:00  0.000095     5.0
14516      108.5      -0.5  2023-04-30 22:00:00       NaN     1.0
14517      108.5      -0.5  2023-04-30 22:00:00  0.000940     5.0
14518      108.5      -0.5  2023-04-30 23:00:00       NaN     1.0
14519      108.5      -0.5  2023-04-30 23:00:00  0.001323     5.0

[14520 rows x 5 columns]


### Interpolate the missing values

In [ ]:
data['tp'] = data['tp'].interpolate(method='quadratic', limit_direction='both')
data['tp'] = data['tp'].clip(lower=0)
#check the nan  values
nan_values = data['tp'].isna().any()
nan_percentage = (data['tp'].isna().sum() / len(data['tp'])) * 100
print((data))

       longitude  latitude                 time        tp  expver
0          108.5      -0.5  2022-01-01 00:00:00  0.000303     NaN
1          108.5      -0.5  2022-01-01 01:00:00  0.000587     NaN
2          108.5      -0.5  2022-01-01 02:00:00  0.000704     NaN
3          108.5      -0.5  2022-01-01 03:00:00  0.000229     NaN
4          108.5      -0.5  2022-01-01 04:00:00  0.000057     NaN
...          ...       ...                  ...       ...     ...
14515      108.5      -0.5  2023-04-30 21:00:00  0.000095     5.0
14516      108.5      -0.5  2023-04-30 22:00:00  0.000499     1.0
14517      108.5      -0.5  2023-04-30 22:00:00  0.000940     5.0
14518      108.5      -0.5  2023-04-30 23:00:00  0.001215     1.0
14519      108.5      -0.5  2023-04-30 23:00:00  0.001323     5.0

[14520 rows x 5 columns]


### Check if there are any nan values

In [ ]:
# Print the result
print("Percentage of NaN values in the Precipitation data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the Precipitation data.")
else:
    print("No NaN values found in the Precipitation data.")
print(data)
# daerah = data

Percentage of NaN values in the Precipitation data: 0.00%
No NaN values found in the Precipitation data.
       longitude  latitude                 time        tp  expver
0          108.5      -0.5  2022-01-01 00:00:00  0.000303     NaN
1          108.5      -0.5  2022-01-01 01:00:00  0.000587     NaN
2          108.5      -0.5  2022-01-01 02:00:00  0.000704     NaN
3          108.5      -0.5  2022-01-01 03:00:00  0.000229     NaN
4          108.5      -0.5  2022-01-01 04:00:00  0.000057     NaN
...          ...       ...                  ...       ...     ...
14515      108.5      -0.5  2023-04-30 21:00:00  0.000095     5.0
14516      108.5      -0.5  2023-04-30 22:00:00  0.000499     1.0
14517      108.5      -0.5  2023-04-30 22:00:00  0.000940     5.0
14518      108.5      -0.5  2023-04-30 23:00:00  0.001215     1.0
14519      108.5      -0.5  2023-04-30 23:00:00  0.001323     5.0

[14520 rows x 5 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[-0.01592451]
 [ 0.3652316 ]
 [ 0.52264763]
 ...
 [ 0.84086337]
 [ 1.20987492]
 [ 1.35474031]]


((11605, 9, 1), (11605, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3872, 3, 1), (5, 3, 1), (484, 3, 1))

In [ ]:
# fitting model to data 
model = define_model()
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
363/363 - 16s - loss: 0.8663 - accuracy: 0.3570 - val_loss: 0.0199 - val_accuracy: 0.2500 - 16s/epoch - 45ms/step
Epoch 2/10
363/363 - 7s - loss: 0.6726 - accuracy: 0.3979 - val_loss: 0.0464 - val_accuracy: 0.2500 - 7s/epoch - 20ms/step
Epoch 3/10
363/363 - 8s - loss: 0.6400 - accuracy: 0.4019 - val_loss: 0.0081 - val_accuracy: 0.2500 - 8s/epoch - 23ms/step
Epoch 4/10
363/363 - 8s - loss: 0.6279 - accuracy: 0.4107 - val_loss: 0.0053 - val_accuracy: 0.2500 - 8s/epoch - 22ms/step
Epoch 5/10
363/363 - 8s - loss: 0.6244 - accuracy: 0.4007 - val_loss: 0.0015 - val_accuracy: 0.2500 - 8s/epoch - 21ms/step
Epoch 6/10
363/363 - 9s - loss: 0.6088 - accuracy: 0.4150 - val_loss: 0.0046 - val_accuracy: 0.2500 - 9s/epoch - 24ms/step
Epoch 7/10
363/363 - 7s - loss: 0.5986 - accuracy: 0.4138 - val_loss: 0.0033 - val_accuracy: 0.2500 - 7s/epoch - 19ms/step
Epoch 8/10
363/363 - 9s - loss: 0.5849 - accuracy: 0.4108 - val_loss: 0.0062 - val_accuracy: 0.2500 - 9s/epoch - 24ms/step
Epoch 9/10
363

In [ ]:
test_results = forecast(data,'WPP711')
test_results

,Predictions,Actuals,Date
0,0.000005,0.000000,2023-03-31 18:00:00
1,0.000060,0.000003,2023-03-31 18:00:00
2,0.000101,0.000007,2023-03-31 19:00:00
3,0.000015,0.000007,2023-03-31 19:00:00
4,0.000068,0.000002,2023-03-31 20:00:00
...,...,...,...
1447,0.000057,0.000095,2023-04-30 21:00:00
1448,0.000105,0.000499,2023-04-30 22:00:00
1449,0.000714,0.000940,2023-04-30 22:00:00
1450,0.000601,0.001215,2023-04-30 23:00:00


In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

,values
rmse,0.00024
corr,0.69020
bias,0.00003
pvalue,0.00160


In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv'+'presipitasiwpp711'+'.h5'
model.save(saved_model_path)

# WPP712

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv/prep712.csv')
data = data.sort_values('time')

data = data.reset_index(drop=True, inplace=False)
print(data)

       longitude  latitude                 time        tp  expver
0          113.0      -6.0  2022-01-01 00:00:00  0.000272     NaN
1          113.0      -6.0  2022-01-01 01:00:00  0.000152     NaN
2          113.0      -6.0  2022-01-01 02:00:00  0.000583     NaN
3          113.0      -6.0  2022-01-01 03:00:00  0.000795     NaN
4          113.0      -6.0  2022-01-01 04:00:00  0.001638     NaN
...          ...       ...                  ...       ...     ...
14515      113.0      -6.0  2023-04-30 21:00:00  0.000024     5.0
14516      113.0      -6.0  2023-04-30 22:00:00       NaN     1.0
14517      113.0      -6.0  2023-04-30 22:00:00  0.000014     5.0
14518      113.0      -6.0  2023-04-30 23:00:00       NaN     1.0
14519      113.0      -6.0  2023-04-30 23:00:00  0.000119     5.0

[14520 rows x 5 columns]


### Interpolate the missing values

In [ ]:
data['tp'] = data['tp'].interpolate(method='quadratic', limit_direction='both')
data['tp'] = data['tp'].clip(lower=0)
#check the nan  values
nan_values = data['tp'].isna().any()
nan_percentage = (data['tp'].isna().sum() / len(data['tp'])) * 100
print((data))

       longitude  latitude                 time        tp  expver
0          113.0      -6.0  2022-01-01 00:00:00  0.000272     NaN
1          113.0      -6.0  2022-01-01 01:00:00  0.000152     NaN
2          113.0      -6.0  2022-01-01 02:00:00  0.000583     NaN
3          113.0      -6.0  2022-01-01 03:00:00  0.000795     NaN
4          113.0      -6.0  2022-01-01 04:00:00  0.001638     NaN
...          ...       ...                  ...       ...     ...
14515      113.0      -6.0  2023-04-30 21:00:00  0.000024     5.0
14516      113.0      -6.0  2023-04-30 22:00:00  0.000013     1.0
14517      113.0      -6.0  2023-04-30 22:00:00  0.000014     5.0
14518      113.0      -6.0  2023-04-30 23:00:00  0.000050     1.0
14519      113.0      -6.0  2023-04-30 23:00:00  0.000119     5.0

[14520 rows x 5 columns]


### Check if there are any nan values

In [ ]:
# Print the result
print("Percentage of NaN values in the Precipitation data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the Precipitation data.")
else:
    print("No NaN values found in the Precipitation data.")
print(data)
# daerah = data

Percentage of NaN values in the Precipitation data: 0.00%
No NaN values found in the Precipitation data.
       longitude  latitude                 time        tp  expver
0          113.0      -6.0  2022-01-01 00:00:00  0.000272     NaN
1          113.0      -6.0  2022-01-01 01:00:00  0.000152     NaN
2          113.0      -6.0  2022-01-01 02:00:00  0.000583     NaN
3          113.0      -6.0  2022-01-01 03:00:00  0.000795     NaN
4          113.0      -6.0  2022-01-01 04:00:00  0.001638     NaN
...          ...       ...                  ...       ...     ...
14515      113.0      -6.0  2023-04-30 21:00:00  0.000024     5.0
14516      113.0      -6.0  2023-04-30 22:00:00  0.000013     1.0
14517      113.0      -6.0  2023-04-30 22:00:00  0.000014     5.0
14518      113.0      -6.0  2023-04-30 23:00:00  0.000050     1.0
14519      113.0      -6.0  2023-04-30 23:00:00  0.000119     5.0

[14520 rows x 5 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[-0.07608985]
 [-0.27423508]
 [ 0.43996102]
 ...
 [-0.50187256]
 [-0.4435835 ]
 [-0.32868086]]


((11605, 9, 1), (11605, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3872, 3, 1), (5, 3, 1), (484, 3, 1))

In [ ]:
# fitting model to data 
model = define_model()
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
363/363 - 17s - loss: 0.7235 - accuracy: 0.3656 - val_loss: 0.0785 - val_accuracy: 0.0000e+00 - 17s/epoch - 46ms/step
Epoch 2/10
363/363 - 9s - loss: 0.5759 - accuracy: 0.3866 - val_loss: 0.1716 - val_accuracy: 0.0000e+00 - 9s/epoch - 24ms/step
Epoch 3/10
363/363 - 7s - loss: 0.5601 - accuracy: 0.3905 - val_loss: 0.1072 - val_accuracy: 0.0000e+00 - 7s/epoch - 19ms/step
Epoch 4/10
363/363 - 8s - loss: 0.5558 - accuracy: 0.3847 - val_loss: 0.0701 - val_accuracy: 0.0000e+00 - 8s/epoch - 23ms/step
Epoch 5/10
363/363 - 7s - loss: 0.5518 - accuracy: 0.3934 - val_loss: 0.0765 - val_accuracy: 0.0000e+00 - 7s/epoch - 20ms/step
Epoch 6/10
363/363 - 8s - loss: 0.5450 - accuracy: 0.3883 - val_loss: 0.1190 - val_accuracy: 0.0000e+00 - 8s/epoch - 22ms/step
Epoch 7/10
363/363 - 8s - loss: 0.5421 - accuracy: 0.3923 - val_loss: 0.0929 - val_accuracy: 0.0000e+00 - 8s/epoch - 23ms/step
Epoch 8/10
363/363 - 7s - loss: 0.5395 - accuracy: 0.3868 - val_loss: 0.1101 - val_accuracy: 0.0000e+00 - 7s/

In [ ]:
test_results = forecast(data,'WPP712')
test_results

,Predictions,Actuals,Date
0,0.000043,0.000793,2023-03-31 18:00:00
1,0.000060,0.000702,2023-03-31 18:00:00
2,0.000103,0.000491,2023-03-31 19:00:00
3,0.000512,0.000395,2023-03-31 19:00:00
4,0.000522,0.000414,2023-03-31 20:00:00
...,...,...,...
1447,0.000087,0.000024,2023-04-30 21:00:00
1448,0.000155,0.000013,2023-04-30 22:00:00
1449,0.000027,0.000014,2023-04-30 22:00:00
1450,0.000070,0.000050,2023-04-30 23:00:00


In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

,values
rmse,0.00028
corr,0.74776
bias,0.00000
pvalue,0.86054


In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv'+'presipitasiwpp712'+'.h5'
model.save(saved_model_path)

# WPP713

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv/prep713.csv')
data = data.sort_values('time')

data = data.reset_index(drop=True, inplace=False)
print(data)

       longitude  latitude                 time        tp  expver
0          119.0      -7.5  2022-01-01 00:00:00  0.000188     NaN
1          119.0      -7.5  2022-01-01 01:00:00  0.000166     NaN
2          119.0      -7.5  2022-01-01 02:00:00  0.000153     NaN
3          119.0      -7.5  2022-01-01 03:00:00  0.000108     NaN
4          119.0      -7.5  2022-01-01 04:00:00  0.000229     NaN
...          ...       ...                  ...       ...     ...
14515      119.0      -7.5  2023-04-30 21:00:00  0.000009     5.0
14516      119.0      -7.5  2023-04-30 22:00:00       NaN     1.0
14517      119.0      -7.5  2023-04-30 22:00:00  0.000015     5.0
14518      119.0      -7.5  2023-04-30 23:00:00       NaN     1.0
14519      119.0      -7.5  2023-04-30 23:00:00  0.000019     5.0

[14520 rows x 5 columns]


### Interpolate the missing values

In [ ]:
data['tp'] = data['tp'].interpolate(method='quadratic', limit_direction='both')
data['tp'] = data['tp'].clip(lower=0)
#check the nan  values
nan_values = data['tp'].isna().any()
nan_percentage = (data['tp'].isna().sum() / len(data['tp'])) * 100
print((data))

       longitude  latitude                 time        tp  expver
0          119.0      -7.5  2022-01-01 00:00:00  0.000188     NaN
1          119.0      -7.5  2022-01-01 01:00:00  0.000166     NaN
2          119.0      -7.5  2022-01-01 02:00:00  0.000153     NaN
3          119.0      -7.5  2022-01-01 03:00:00  0.000108     NaN
4          119.0      -7.5  2022-01-01 04:00:00  0.000229     NaN
...          ...       ...                  ...       ...     ...
14515      119.0      -7.5  2023-04-30 21:00:00  0.000009     5.0
14516      119.0      -7.5  2023-04-30 22:00:00  0.000012     1.0
14517      119.0      -7.5  2023-04-30 22:00:00  0.000015     5.0
14518      119.0      -7.5  2023-04-30 23:00:00  0.000017     1.0
14519      119.0      -7.5  2023-04-30 23:00:00  0.000019     5.0

[14520 rows x 5 columns]


### Check if there are any nan values

In [ ]:
# Print the result
print("Percentage of NaN values in the Precipitation data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the Precipitation data.")
else:
    print("No NaN values found in the Precipitation data.")
print(data)
# daerah = data

Percentage of NaN values in the Precipitation data: 0.00%
No NaN values found in the Precipitation data.
       longitude  latitude                 time        tp  expver
0          119.0      -7.5  2022-01-01 00:00:00  0.000188     NaN
1          119.0      -7.5  2022-01-01 01:00:00  0.000166     NaN
2          119.0      -7.5  2022-01-01 02:00:00  0.000153     NaN
3          119.0      -7.5  2022-01-01 03:00:00  0.000108     NaN
4          119.0      -7.5  2022-01-01 04:00:00  0.000229     NaN
...          ...       ...                  ...       ...     ...
14515      119.0      -7.5  2023-04-30 21:00:00  0.000009     5.0
14516      119.0      -7.5  2023-04-30 22:00:00  0.000012     1.0
14517      119.0      -7.5  2023-04-30 22:00:00  0.000015     5.0
14518      119.0      -7.5  2023-04-30 23:00:00  0.000017     1.0
14519      119.0      -7.5  2023-04-30 23:00:00  0.000019     5.0

[14520 rows x 5 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[-0.13590657]
 [-0.17899485]
 [-0.20307473]
 ...
 [-0.46938572]
 [-0.46525467]
 [-0.46214273]]


((11605, 9, 1), (11605, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3872, 3, 1), (5, 3, 1), (484, 3, 1))

In [ ]:
# fitting model to data 
model = define_model()
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
363/363 - 16s - loss: 0.6688 - accuracy: 0.3383 - val_loss: 0.2745 - val_accuracy: 0.7500 - 16s/epoch - 45ms/step
Epoch 2/10
363/363 - 15s - loss: 0.5402 - accuracy: 0.3759 - val_loss: 0.2481 - val_accuracy: 0.7500 - 15s/epoch - 40ms/step
Epoch 3/10
363/363 - 13s - loss: 0.5232 - accuracy: 0.3822 - val_loss: 0.2435 - val_accuracy: 0.2500 - 13s/epoch - 37ms/step
Epoch 4/10
363/363 - 7s - loss: 0.5115 - accuracy: 0.3851 - val_loss: 0.2354 - val_accuracy: 0.2500 - 7s/epoch - 20ms/step
Epoch 5/10
363/363 - 9s - loss: 0.5076 - accuracy: 0.3853 - val_loss: 0.2143 - val_accuracy: 0.7500 - 9s/epoch - 24ms/step
Epoch 6/10
363/363 - 8s - loss: 0.5019 - accuracy: 0.3841 - val_loss: 0.2091 - val_accuracy: 0.7500 - 8s/epoch - 22ms/step
Epoch 7/10
363/363 - 9s - loss: 0.5024 - accuracy: 0.3826 - val_loss: 0.2393 - val_accuracy: 0.2500 - 9s/epoch - 24ms/step
Epoch 8/10
363/363 - 10s - loss: 0.4940 - accuracy: 0.3845 - val_loss: 0.2156 - val_accuracy: 0.5000 - 10s/epoch - 29ms/step
Epoch 9/

In [ ]:
test_results = forecast(data,'WPP713')
test_results

,Predictions,Actuals,Date
0,0.000393,0.001022,2023-03-31 18:00:00
1,0.000405,0.001112,2023-03-31 18:00:00
2,0.000384,0.001130,2023-03-31 19:00:00
3,0.000946,0.000871,2023-03-31 19:00:00
4,0.000842,0.000384,2023-03-31 20:00:00
...,...,...,...
1447,0.000035,0.000009,2023-04-30 21:00:00
1448,0.000058,0.000012,2023-04-30 22:00:00
1449,0.000023,0.000015,2023-04-30 22:00:00
1450,0.000047,0.000017,2023-04-30 23:00:00


In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

,values
rmse,0.00030
corr,0.79930
bias,-0.00001
pvalue,0.41835


In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv'+'presipitasiwpp713'+'.h5'
model.save(saved_model_path)

# WPP714


In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv/prep714.csv')
data = data.sort_values('time')

data = data.reset_index(drop=True, inplace=False)
print(data)

       longitude  latitude                 time            tp  expver
0          127.5      -7.5  2022-01-01 00:00:00  1.644157e-05     NaN
1          127.5      -7.5  2022-01-01 01:00:00  7.232651e-06     NaN
2          127.5      -7.5  2022-01-01 02:00:00  2.630055e-06     NaN
3          127.5      -7.5  2022-01-01 03:00:00 -1.862645e-09     NaN
4          127.5      -7.5  2022-01-01 04:00:00  6.556511e-07     NaN
...          ...       ...                  ...           ...     ...
14515      127.5      -7.5  2023-04-30 21:00:00  2.173968e-04     5.0
14516      127.5      -7.5  2023-04-30 22:00:00           NaN     1.0
14517      127.5      -7.5  2023-04-30 22:00:00  1.052525e-04     5.0
14518      127.5      -7.5  2023-04-30 23:00:00           NaN     1.0
14519      127.5      -7.5  2023-04-30 23:00:00  9.836070e-05     5.0

[14520 rows x 5 columns]


### Interpolate the missing values

In [ ]:
data['tp'] = data['tp'].interpolate(method='quadratic', limit_direction='both')
data['tp'] = data['tp'].clip(lower=0)
#check the nan  values
nan_values = data['tp'].isna().any()
nan_percentage = (data['tp'].isna().sum() / len(data['tp'])) * 100
print((data))

       longitude  latitude                 time            tp  expver
0          127.5      -7.5  2022-01-01 00:00:00  1.644157e-05     NaN
1          127.5      -7.5  2022-01-01 01:00:00  7.232651e-06     NaN
2          127.5      -7.5  2022-01-01 02:00:00  2.630055e-06     NaN
3          127.5      -7.5  2022-01-01 03:00:00  0.000000e+00     NaN
4          127.5      -7.5  2022-01-01 04:00:00  6.556511e-07     NaN
...          ...       ...                  ...           ...     ...
14515      127.5      -7.5  2023-04-30 21:00:00  2.173968e-04     5.0
14516      127.5      -7.5  2023-04-30 22:00:00  1.593805e-04     1.0
14517      127.5      -7.5  2023-04-30 22:00:00  1.052525e-04     5.0
14518      127.5      -7.5  2023-04-30 23:00:00  8.491255e-05     1.0
14519      127.5      -7.5  2023-04-30 23:00:00  9.836070e-05     5.0

[14520 rows x 5 columns]


### Check if there are any nan values

In [ ]:
# Print the result
print("Percentage of NaN values in the Precipitation data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the Precipitation data.")
else:
    print("No NaN values found in the Precipitation data.")
print(data)
# daerah = data

Percentage of NaN values in the Precipitation data: 0.00%
No NaN values found in the Precipitation data.
       longitude  latitude                 time            tp  expver
0          127.5      -7.5  2022-01-01 00:00:00  1.644157e-05     NaN
1          127.5      -7.5  2022-01-01 01:00:00  7.232651e-06     NaN
2          127.5      -7.5  2022-01-01 02:00:00  2.630055e-06     NaN
3          127.5      -7.5  2022-01-01 03:00:00  0.000000e+00     NaN
4          127.5      -7.5  2022-01-01 04:00:00  6.556511e-07     NaN
...          ...       ...                  ...           ...     ...
14515      127.5      -7.5  2023-04-30 21:00:00  2.173968e-04     5.0
14516      127.5      -7.5  2023-04-30 22:00:00  1.593805e-04     1.0
14517      127.5      -7.5  2023-04-30 22:00:00  1.052525e-04     5.0
14518      127.5      -7.5  2023-04-30 23:00:00  8.491255e-05     1.0
14519      127.5      -7.5  2023-04-30 23:00:00  9.836070e-05     5.0

[14520 rows x 5 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[-0.35153495]
 [-0.36769464]
 [-0.37577121]
 ...
 [-0.19569067]
 [-0.23138295]
 [-0.2077843 ]]


((11605, 9, 1), (11605, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3872, 3, 1), (5, 3, 1), (484, 3, 1))

In [ ]:
# fitting model to data 
model = define_model()
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
363/363 - 16s - loss: 0.5037 - accuracy: 0.3301 - val_loss: 0.2560 - val_accuracy: 0.7500 - 16s/epoch - 45ms/step
Epoch 2/10
363/363 - 11s - loss: 0.4077 - accuracy: 0.3639 - val_loss: 0.1416 - val_accuracy: 0.7500 - 11s/epoch - 30ms/step
Epoch 3/10
363/363 - 9s - loss: 0.3866 - accuracy: 0.3856 - val_loss: 0.1297 - val_accuracy: 0.7500 - 9s/epoch - 23ms/step
Epoch 4/10
363/363 - 7s - loss: 0.3856 - accuracy: 0.3878 - val_loss: 0.1326 - val_accuracy: 0.7500 - 7s/epoch - 19ms/step
Epoch 5/10
363/363 - 9s - loss: 0.3817 - accuracy: 0.3856 - val_loss: 0.1417 - val_accuracy: 0.7500 - 9s/epoch - 24ms/step
Epoch 6/10
363/363 - 7s - loss: 0.3795 - accuracy: 0.3793 - val_loss: 0.1624 - val_accuracy: 0.7500 - 7s/epoch - 19ms/step
Epoch 7/10
363/363 - 8s - loss: 0.3785 - accuracy: 0.3866 - val_loss: 0.1311 - val_accuracy: 0.7500 - 8s/epoch - 23ms/step
Epoch 8/10
363/363 - 7s - loss: 0.3727 - accuracy: 0.3903 - val_loss: 0.1495 - val_accuracy: 0.5000 - 7s/epoch - 20ms/step
Epoch 9/10
3

In [ ]:
test_results = forecast(data,'WPP714')
test_results

,Predictions,Actuals,Date
0,0.000057,0.000000,2023-03-31 18:00:00
1,0.000097,0.000000,2023-03-31 18:00:00
2,0.000118,0.000002,2023-03-31 19:00:00
3,0.000053,0.000004,2023-03-31 19:00:00
4,0.000093,0.000005,2023-03-31 20:00:00
...,...,...,...
1447,0.000110,0.000003,2023-04-30 21:00:00
1448,0.000143,0.000034,2023-04-30 22:00:00
1449,0.000096,0.000057,2023-04-30 22:00:00
1450,0.000125,0.000054,2023-04-30 23:00:00


In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

,values
rmse,0.00034
corr,0.86332
bias,0.00000
pvalue,0.84539


In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv'+'presipitasiwpp714'+'.h5'
model.save(saved_model_path)

# WPP715


In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv/prep715.csv')
data = data.sort_values('time')

data = data.reset_index(drop=True, inplace=False)
print(data)

       longitude  latitude                 time        tp  expver
0          127.0      -1.5  2022-01-01 00:00:00  0.002542     NaN
1          127.0      -1.5  2022-01-01 01:00:00  0.001518     NaN
2          127.0      -1.5  2022-01-01 02:00:00  0.000216     NaN
3          127.0      -1.5  2022-01-01 03:00:00  0.000108     NaN
4          127.0      -1.5  2022-01-01 04:00:00  0.000076     NaN
...          ...       ...                  ...       ...     ...
14515      127.0      -1.5  2023-04-30 21:00:00  0.000020     5.0
14516      127.0      -1.5  2023-04-30 22:00:00       NaN     1.0
14517      127.0      -1.5  2023-04-30 22:00:00  0.000165     5.0
14518      127.0      -1.5  2023-04-30 23:00:00       NaN     1.0
14519      127.0      -1.5  2023-04-30 23:00:00  0.000491     5.0

[14520 rows x 5 columns]


### Interpolate the missing values

In [ ]:
data['tp'] = data['tp'].interpolate(method='quadratic', limit_direction='both')
data['tp'] = data['tp'].clip(lower=0)
#check the nan  values
nan_values = data['tp'].isna().any()
nan_percentage = (data['tp'].isna().sum() / len(data['tp'])) * 100
print((data))

       longitude  latitude                 time        tp  expver
0          127.0      -1.5  2022-01-01 00:00:00  0.002542     NaN
1          127.0      -1.5  2022-01-01 01:00:00  0.001518     NaN
2          127.0      -1.5  2022-01-01 02:00:00  0.000216     NaN
3          127.0      -1.5  2022-01-01 03:00:00  0.000108     NaN
4          127.0      -1.5  2022-01-01 04:00:00  0.000076     NaN
...          ...       ...                  ...       ...     ...
14515      127.0      -1.5  2023-04-30 21:00:00  0.000020     5.0
14516      127.0      -1.5  2023-04-30 22:00:00  0.000071     1.0
14517      127.0      -1.5  2023-04-30 22:00:00  0.000165     5.0
14518      127.0      -1.5  2023-04-30 23:00:00  0.000305     1.0
14519      127.0      -1.5  2023-04-30 23:00:00  0.000491     5.0

[14520 rows x 5 columns]


### Check if there are any nan values

In [ ]:
# Print the result
print("Percentage of NaN values in the Precipitation data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the Precipitation data.")
else:
    print("No NaN values found in the Precipitation data.")
print(data)
# daerah = data

Percentage of NaN values in the Precipitation data: 0.00%
No NaN values found in the Precipitation data.
       longitude  latitude                 time        tp  expver
0          127.0      -1.5  2022-01-01 00:00:00  0.002542     NaN
1          127.0      -1.5  2022-01-01 01:00:00  0.001518     NaN
2          127.0      -1.5  2022-01-01 02:00:00  0.000216     NaN
3          127.0      -1.5  2022-01-01 03:00:00  0.000108     NaN
4          127.0      -1.5  2022-01-01 04:00:00  0.000076     NaN
...          ...       ...                  ...       ...     ...
14515      127.0      -1.5  2023-04-30 21:00:00  0.000020     5.0
14516      127.0      -1.5  2023-04-30 22:00:00  0.000071     1.0
14517      127.0      -1.5  2023-04-30 22:00:00  0.000165     5.0
14518      127.0      -1.5  2023-04-30 23:00:00  0.000305     1.0
14519      127.0      -1.5  2023-04-30 23:00:00  0.000491     5.0

[14520 rows x 5 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[ 3.21150452]
 [ 1.72097886]
 [-0.17448747]
 ...
 [-0.24864938]
 [-0.04499421]
 [ 0.22645689]]


((11605, 9, 1), (11605, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3872, 3, 1), (5, 3, 1), (484, 3, 1))

In [ ]:
# fitting model to data 
model = define_model()
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
363/363 - 17s - loss: 0.6981 - accuracy: 0.3808 - val_loss: 0.0850 - val_accuracy: 0.5000 - 17s/epoch - 46ms/step
Epoch 2/10
363/363 - 8s - loss: 0.5162 - accuracy: 0.4156 - val_loss: 0.0553 - val_accuracy: 0.5000 - 8s/epoch - 22ms/step
Epoch 3/10
363/363 - 8s - loss: 0.4950 - accuracy: 0.4253 - val_loss: 0.0660 - val_accuracy: 0.5000 - 8s/epoch - 22ms/step
Epoch 4/10
363/363 - 8s - loss: 0.4887 - accuracy: 0.4302 - val_loss: 0.0391 - val_accuracy: 0.5000 - 8s/epoch - 21ms/step
Epoch 5/10
363/363 - 8s - loss: 0.4775 - accuracy: 0.4425 - val_loss: 0.0603 - val_accuracy: 0.5000 - 8s/epoch - 23ms/step
Epoch 6/10
363/363 - 7s - loss: 0.4779 - accuracy: 0.4341 - val_loss: 0.0678 - val_accuracy: 0.5000 - 7s/epoch - 20ms/step
Epoch 7/10
363/363 - 8s - loss: 0.4732 - accuracy: 0.4448 - val_loss: 0.0528 - val_accuracy: 0.5000 - 8s/epoch - 23ms/step
Epoch 8/10
363/363 - 7s - loss: 0.4736 - accuracy: 0.4425 - val_loss: 0.0617 - val_accuracy: 0.5000 - 7s/epoch - 19ms/step
Epoch 9/10
363

In [ ]:
test_results = forecast(data,'WPP715')
test_results

,Predictions,Actuals,Date
0,0.000026,0.000861,2023-03-31 18:00:00
1,0.000077,0.000825,2023-03-31 18:00:00
2,0.000118,0.000855,2023-03-31 19:00:00
3,0.000790,0.000935,2023-03-31 19:00:00
4,0.000657,0.000999,2023-03-31 20:00:00
...,...,...,...
1447,0.000031,0.000020,2023-04-30 21:00:00
1448,0.000057,0.000071,2023-04-30 22:00:00
1449,0.000095,0.000165,2023-04-30 22:00:00
1450,0.000091,0.000305,2023-04-30 23:00:00


In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

,values
rmse,0.00039
corr,0.81211
bias,-0.00002
pvalue,0.40433


In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv'+'presipitasiwpp715'+'.h5'
model.save(saved_model_path)

# WPP716

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv/prep716.csv')
data = data.sort_values('time')

data = data.reset_index(drop=True, inplace=False)
print(data)

       longitude  latitude                 time        tp  expver
0          125.0       1.5  2022-01-01 00:00:00  0.000048     NaN
1          125.0       1.5  2022-01-01 01:00:00  0.000114     NaN
2          125.0       1.5  2022-01-01 02:00:00  0.001940     NaN
3          125.0       1.5  2022-01-01 03:00:00  0.001114     NaN
4          125.0       1.5  2022-01-01 04:00:00  0.000253     NaN
...          ...       ...                  ...       ...     ...
14515      125.0       1.5  2023-04-30 21:00:00  0.000061     5.0
14516      125.0       1.5  2023-04-30 22:00:00       NaN     1.0
14517      125.0       1.5  2023-04-30 22:00:00  0.000030     5.0
14518      125.0       1.5  2023-04-30 23:00:00       NaN     1.0
14519      125.0       1.5  2023-04-30 23:00:00  0.000111     5.0

[14520 rows x 5 columns]


### Interpolate the missing values

In [ ]:
data['tp'] = data['tp'].interpolate(method='quadratic', limit_direction='both')
data['tp'] = data['tp'].clip(lower=0)
#check the nan  values
nan_values = data['tp'].isna().any()
nan_percentage = (data['tp'].isna().sum() / len(data['tp'])) * 100
print((data))

       longitude  latitude                 time        tp  expver
0          125.0       1.5  2022-01-01 00:00:00  0.000048     NaN
1          125.0       1.5  2022-01-01 01:00:00  0.000114     NaN
2          125.0       1.5  2022-01-01 02:00:00  0.001940     NaN
3          125.0       1.5  2022-01-01 03:00:00  0.001114     NaN
4          125.0       1.5  2022-01-01 04:00:00  0.000253     NaN
...          ...       ...                  ...       ...     ...
14515      125.0       1.5  2023-04-30 21:00:00  0.000061     5.0
14516      125.0       1.5  2023-04-30 22:00:00  0.000073     1.0
14517      125.0       1.5  2023-04-30 22:00:00  0.000030     5.0
14518      125.0       1.5  2023-04-30 23:00:00  0.000043     1.0
14519      125.0       1.5  2023-04-30 23:00:00  0.000111     5.0

[14520 rows x 5 columns]


### Check if there are any nan values

In [ ]:
# Print the result
print("Percentage of NaN values in the Precipitation data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the Precipitation data.")
else:
    print("No NaN values found in the Precipitation data.")
print(data)
# daerah = data

Percentage of NaN values in the Precipitation data: 0.00%
No NaN values found in the Precipitation data.
       longitude  latitude                 time        tp  expver
0          125.0       1.5  2022-01-01 00:00:00  0.000048     NaN
1          125.0       1.5  2022-01-01 01:00:00  0.000114     NaN
2          125.0       1.5  2022-01-01 02:00:00  0.001940     NaN
3          125.0       1.5  2022-01-01 03:00:00  0.001114     NaN
4          125.0       1.5  2022-01-01 04:00:00  0.000253     NaN
...          ...       ...                  ...       ...     ...
14515      125.0       1.5  2023-04-30 21:00:00  0.000061     5.0
14516      125.0       1.5  2023-04-30 22:00:00  0.000073     1.0
14517      125.0       1.5  2023-04-30 22:00:00  0.000030     5.0
14518      125.0       1.5  2023-04-30 23:00:00  0.000043     1.0
14519      125.0       1.5  2023-04-30 23:00:00  0.000111     5.0

[14520 rows x 5 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[-0.41203213]
 [-0.34626249]
 [ 1.48012923]
 ...
 [-0.42996931]
 [-0.41747987]
 [-0.34915144]]


((11605, 9, 1), (11605, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3872, 3, 1), (5, 3, 1), (484, 3, 1))

In [ ]:
# fitting model to data 
model = define_model()
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
363/363 - 16s - loss: 0.7557 - accuracy: 0.4149 - val_loss: 0.2065 - val_accuracy: 0.5000 - 16s/epoch - 45ms/step
Epoch 2/10
363/363 - 7s - loss: 0.6676 - accuracy: 0.4309 - val_loss: 0.2194 - val_accuracy: 0.2500 - 7s/epoch - 20ms/step
Epoch 3/10
363/363 - 8s - loss: 0.6482 - accuracy: 0.4323 - val_loss: 0.2125 - val_accuracy: 0.2500 - 8s/epoch - 23ms/step
Epoch 4/10
363/363 - 8s - loss: 0.6448 - accuracy: 0.4377 - val_loss: 0.2691 - val_accuracy: 0.5000 - 8s/epoch - 22ms/step
Epoch 5/10
363/363 - 7s - loss: 0.6308 - accuracy: 0.4421 - val_loss: 0.2546 - val_accuracy: 0.5000 - 7s/epoch - 20ms/step
Epoch 6/10
363/363 - 8s - loss: 0.6296 - accuracy: 0.4438 - val_loss: 0.2203 - val_accuracy: 0.5000 - 8s/epoch - 23ms/step
Epoch 7/10
363/363 - 7s - loss: 0.6302 - accuracy: 0.4489 - val_loss: 0.2038 - val_accuracy: 0.5000 - 7s/epoch - 19ms/step
Epoch 8/10
363/363 - 8s - loss: 0.6160 - accuracy: 0.4444 - val_loss: 0.2766 - val_accuracy: 0.2500 - 8s/epoch - 23ms/step
Epoch 9/10
363

In [ ]:
test_results = forecast(data,'WPP716')
test_results

,Predictions,Actuals,Date
0,0.000843,0.000021,2023-03-31 18:00:00
1,0.000856,0.000000,2023-03-31 18:00:00
2,0.000819,0.000000,2023-03-31 19:00:00
3,0.000013,0.000222,2023-03-31 19:00:00
4,0.000084,0.000526,2023-03-31 20:00:00
...,...,...,...
1447,0.000285,0.000061,2023-04-30 21:00:00
1448,0.000302,0.000073,2023-04-30 22:00:00
1449,0.000112,0.000030,2023-04-30 22:00:00
1450,0.000165,0.000043,2023-04-30 23:00:00


In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

,values
rmse,0.00041
corr,0.74068
bias,0.00009
pvalue,0.00000


In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv'+'presipitasiwpp716'+'.h5'
model.save(saved_model_path)

# WPP716

In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv/prep716.csv')
data = data.sort_values('time')

data = data.reset_index(drop=True, inplace=False)
print(data)

       longitude  latitude                 time        tp  expver
0          125.0       1.5  2022-01-01 00:00:00  0.000048     NaN
1          125.0       1.5  2022-01-01 01:00:00  0.000114     NaN
2          125.0       1.5  2022-01-01 02:00:00  0.001940     NaN
3          125.0       1.5  2022-01-01 03:00:00  0.001114     NaN
4          125.0       1.5  2022-01-01 04:00:00  0.000253     NaN
...          ...       ...                  ...       ...     ...
14515      125.0       1.5  2023-04-30 21:00:00  0.000061     5.0
14516      125.0       1.5  2023-04-30 22:00:00       NaN     1.0
14517      125.0       1.5  2023-04-30 22:00:00  0.000030     5.0
14518      125.0       1.5  2023-04-30 23:00:00       NaN     1.0
14519      125.0       1.5  2023-04-30 23:00:00  0.000111     5.0

[14520 rows x 5 columns]


### Interpolate the missing values

In [ ]:
data['tp'] = data['tp'].interpolate(method='quadratic', limit_direction='both')
data['tp'] = data['tp'].clip(lower=0)
#check the nan  values
nan_values = data['tp'].isna().any()
nan_percentage = (data['tp'].isna().sum() / len(data['tp'])) * 100
print((data))

       longitude  latitude                 time        tp  expver
0          125.0       1.5  2022-01-01 00:00:00  0.000048     NaN
1          125.0       1.5  2022-01-01 01:00:00  0.000114     NaN
2          125.0       1.5  2022-01-01 02:00:00  0.001940     NaN
3          125.0       1.5  2022-01-01 03:00:00  0.001114     NaN
4          125.0       1.5  2022-01-01 04:00:00  0.000253     NaN
...          ...       ...                  ...       ...     ...
14515      125.0       1.5  2023-04-30 21:00:00  0.000061     5.0
14516      125.0       1.5  2023-04-30 22:00:00  0.000073     1.0
14517      125.0       1.5  2023-04-30 22:00:00  0.000030     5.0
14518      125.0       1.5  2023-04-30 23:00:00  0.000043     1.0
14519      125.0       1.5  2023-04-30 23:00:00  0.000111     5.0

[14520 rows x 5 columns]


### Check if there are any nan values

In [ ]:
# Print the result
print("Percentage of NaN values in the Precipitation data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the Precipitation data.")
else:
    print("No NaN values found in the Precipitation data.")
print(data)
# daerah = data

Percentage of NaN values in the Precipitation data: 0.00%
No NaN values found in the Precipitation data.
       longitude  latitude                 time        tp  expver
0          125.0       1.5  2022-01-01 00:00:00  0.000048     NaN
1          125.0       1.5  2022-01-01 01:00:00  0.000114     NaN
2          125.0       1.5  2022-01-01 02:00:00  0.001940     NaN
3          125.0       1.5  2022-01-01 03:00:00  0.001114     NaN
4          125.0       1.5  2022-01-01 04:00:00  0.000253     NaN
...          ...       ...                  ...       ...     ...
14515      125.0       1.5  2023-04-30 21:00:00  0.000061     5.0
14516      125.0       1.5  2023-04-30 22:00:00  0.000073     1.0
14517      125.0       1.5  2023-04-30 22:00:00  0.000030     5.0
14518      125.0       1.5  2023-04-30 23:00:00  0.000043     1.0
14519      125.0       1.5  2023-04-30 23:00:00  0.000111     5.0

[14520 rows x 5 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[-0.41203213]
 [-0.34626249]
 [ 1.48012923]
 ...
 [-0.42996931]
 [-0.41747987]
 [-0.34915144]]


((11605, 9, 1), (11605, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3872, 3, 1), (5, 3, 1), (484, 3, 1))

In [ ]:
# fitting model to data 
model = define_model()
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
363/363 - 17s - loss: 0.7666 - accuracy: 0.4216 - val_loss: 0.2059 - val_accuracy: 0.2500 - 17s/epoch - 46ms/step
Epoch 2/10
363/363 - 8s - loss: 0.6609 - accuracy: 0.4457 - val_loss: 0.1864 - val_accuracy: 0.2500 - 8s/epoch - 23ms/step
Epoch 3/10
363/363 - 7s - loss: 0.6443 - accuracy: 0.4290 - val_loss: 0.2104 - val_accuracy: 0.2500 - 7s/epoch - 20ms/step
Epoch 4/10
363/363 - 8s - loss: 0.6412 - accuracy: 0.4428 - val_loss: 0.2374 - val_accuracy: 0.5000 - 8s/epoch - 23ms/step
Epoch 5/10
363/363 - 9s - loss: 0.6399 - accuracy: 0.4412 - val_loss: 0.2154 - val_accuracy: 0.2500 - 9s/epoch - 24ms/step
Epoch 6/10
363/363 - 7s - loss: 0.6346 - accuracy: 0.4450 - val_loss: 0.2421 - val_accuracy: 0.5000 - 7s/epoch - 19ms/step
Epoch 7/10
363/363 - 9s - loss: 0.6222 - accuracy: 0.4456 - val_loss: 0.2235 - val_accuracy: 0.5000 - 9s/epoch - 24ms/step
Epoch 8/10
363/363 - 7s - loss: 0.6219 - accuracy: 0.4383 - val_loss: 0.2223 - val_accuracy: 0.2500 - 7s/epoch - 19ms/step
Epoch 9/10
363

In [ ]:
test_results = forecast(data,'WPP716')
test_results

,Predictions,Actuals,Date
0,0.000712,0.000021,2023-03-31 18:00:00
1,0.000776,0.000000,2023-03-31 18:00:00
2,0.000817,0.000000,2023-03-31 19:00:00
3,0.000045,0.000222,2023-03-31 19:00:00
4,0.000145,0.000526,2023-03-31 20:00:00
...,...,...,...
1447,0.000200,0.000061,2023-04-30 21:00:00
1448,0.000260,0.000073,2023-04-30 22:00:00
1449,0.000156,0.000030,2023-04-30 22:00:00
1450,0.000246,0.000043,2023-04-30 23:00:00


In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

,values
rmse,0.00040
corr,0.75663
bias,0.00012
pvalue,0.00000


In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv'+'presipitasiwpp716'+'.h5'
model.save(saved_model_path)

# WPP717


In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv/prep717.csv')
data = data.sort_values('time')

data = data.reset_index(drop=True, inplace=False)
print(data)

       longitude  latitude                 time        tp  expver
0          137.0       0.0  2022-01-01 00:00:00  0.000308     NaN
1          137.0       0.0  2022-01-01 01:00:00  0.000147     NaN
2          137.0       0.0  2022-01-01 02:00:00  0.000012     NaN
3          137.0       0.0  2022-01-01 03:00:00  0.000070     NaN
4          137.0       0.0  2022-01-01 04:00:00  0.000388     NaN
...          ...       ...                  ...       ...     ...
14515      137.0       0.0  2023-04-30 21:00:00  0.000003     5.0
14516      137.0       0.0  2023-04-30 22:00:00       NaN     1.0
14517      137.0       0.0  2023-04-30 22:00:00  0.000057     5.0
14518      137.0       0.0  2023-04-30 23:00:00       NaN     1.0
14519      137.0       0.0  2023-04-30 23:00:00  0.000024     5.0

[14520 rows x 5 columns]


### Interpolate the missing values

In [ ]:
data['tp'] = data['tp'].interpolate(method='quadratic', limit_direction='both')
data['tp'] = data['tp'].clip(lower=0)
#check the nan  values
nan_values = data['tp'].isna().any()
nan_percentage = (data['tp'].isna().sum() / len(data['tp'])) * 100
print((data))

       longitude  latitude                 time        tp  expver
0          137.0       0.0  2022-01-01 00:00:00  0.000308     NaN
1          137.0       0.0  2022-01-01 01:00:00  0.000147     NaN
2          137.0       0.0  2022-01-01 02:00:00  0.000012     NaN
3          137.0       0.0  2022-01-01 03:00:00  0.000070     NaN
4          137.0       0.0  2022-01-01 04:00:00  0.000388     NaN
...          ...       ...                  ...       ...     ...
14515      137.0       0.0  2023-04-30 21:00:00  0.000003     5.0
14516      137.0       0.0  2023-04-30 22:00:00  0.000034     1.0
14517      137.0       0.0  2023-04-30 22:00:00  0.000057     5.0
14518      137.0       0.0  2023-04-30 23:00:00  0.000054     1.0
14519      137.0       0.0  2023-04-30 23:00:00  0.000024     5.0

[14520 rows x 5 columns]


### Check if there are any nan values

In [ ]:
# Print the result
print("Percentage of NaN values in the Precipitation data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the Precipitation data.")
else:
    print("No NaN values found in the Precipitation data.")
print(data)
# daerah = data

Percentage of NaN values in the Precipitation data: 0.00%
No NaN values found in the Precipitation data.
       longitude  latitude                 time        tp  expver
0          137.0       0.0  2022-01-01 00:00:00  0.000308     NaN
1          137.0       0.0  2022-01-01 01:00:00  0.000147     NaN
2          137.0       0.0  2022-01-01 02:00:00  0.000012     NaN
3          137.0       0.0  2022-01-01 03:00:00  0.000070     NaN
4          137.0       0.0  2022-01-01 04:00:00  0.000388     NaN
...          ...       ...                  ...       ...     ...
14515      137.0       0.0  2023-04-30 21:00:00  0.000003     5.0
14516      137.0       0.0  2023-04-30 22:00:00  0.000034     1.0
14517      137.0       0.0  2023-04-30 22:00:00  0.000057     5.0
14518      137.0       0.0  2023-04-30 23:00:00  0.000054     1.0
14519      137.0       0.0  2023-04-30 23:00:00  0.000024     5.0

[14520 rows x 5 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[ 0.06229887]
 [-0.21575537]
 [-0.4463367 ]
 ...
 [-0.36982945]
 [-0.3757594 ]
 [-0.42689617]]


((11605, 9, 1), (11605, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3872, 3, 1), (5, 3, 1), (484, 3, 1))

In [ ]:
# fitting model to data 
model = define_model()
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
363/363 - 17s - loss: 0.7456 - accuracy: 0.3748 - val_loss: 0.0225 - val_accuracy: 0.2500 - 17s/epoch - 46ms/step
Epoch 2/10
363/363 - 7s - loss: 0.6043 - accuracy: 0.4146 - val_loss: 0.0299 - val_accuracy: 0.2500 - 7s/epoch - 19ms/step
Epoch 3/10
363/363 - 9s - loss: 0.5612 - accuracy: 0.4202 - val_loss: 0.0135 - val_accuracy: 0.2500 - 9s/epoch - 24ms/step
Epoch 4/10
363/363 - 8s - loss: 0.5601 - accuracy: 0.4403 - val_loss: 0.0241 - val_accuracy: 0.2500 - 8s/epoch - 21ms/step
Epoch 5/10
363/363 - 8s - loss: 0.5407 - accuracy: 0.4421 - val_loss: 0.0157 - val_accuracy: 0.2500 - 8s/epoch - 23ms/step
Epoch 6/10
363/363 - 8s - loss: 0.5313 - accuracy: 0.4396 - val_loss: 0.0137 - val_accuracy: 0.2500 - 8s/epoch - 23ms/step
Epoch 7/10
363/363 - 7s - loss: 0.5267 - accuracy: 0.4363 - val_loss: 0.0131 - val_accuracy: 0.2500 - 7s/epoch - 19ms/step
Epoch 8/10
363/363 - 8s - loss: 0.5299 - accuracy: 0.4430 - val_loss: 0.0163 - val_accuracy: 0.2500 - 8s/epoch - 23ms/step
Epoch 9/10
363

In [ ]:
test_results = forecast(data,'WPP717')
test_results

,Predictions,Actuals,Date
0,0.000057,0.000000,2023-03-31 18:00:00
1,0.000097,0.000000,2023-03-31 18:00:00
2,0.000118,0.000002,2023-03-31 19:00:00
3,0.000053,0.000004,2023-03-31 19:00:00
4,0.000093,0.000005,2023-03-31 20:00:00
...,...,...,...
1447,0.000110,0.000003,2023-04-30 21:00:00
1448,0.000143,0.000034,2023-04-30 22:00:00
1449,0.000096,0.000057,2023-04-30 22:00:00
1450,0.000125,0.000054,2023-04-30 23:00:00


In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

,values
rmse,0.00034
corr,0.86332
bias,0.00000
pvalue,0.84539


In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv'+'presipitasiwpp717'+'.h5'
model.save(saved_model_path)

# WPP718


In [ ]:
# read data
data = pd.read_csv(r'/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv/prep718.csv')
data = data.sort_values('time')

data = data.reset_index(drop=True, inplace=False)
print(data)

       longitude  latitude                 time        tp  expver
0          136.5      -7.5  2022-01-01 00:00:00  0.000095     NaN
1          136.5      -7.5  2022-01-01 01:00:00  0.000165     NaN
2          136.5      -7.5  2022-01-01 02:00:00  0.000260     NaN
3          136.5      -7.5  2022-01-01 03:00:00  0.000374     NaN
4          136.5      -7.5  2022-01-01 04:00:00  0.000357     NaN
...          ...       ...                  ...       ...     ...
14515      136.5      -7.5  2023-04-30 21:00:00  0.001546     5.0
14516      136.5      -7.5  2023-04-30 22:00:00       NaN     1.0
14517      136.5      -7.5  2023-04-30 22:00:00  0.000890     5.0
14518      136.5      -7.5  2023-04-30 23:00:00       NaN     1.0
14519      136.5      -7.5  2023-04-30 23:00:00  0.000415     5.0

[14520 rows x 5 columns]


### Interpolate the missing values

In [ ]:
data['tp'] = data['tp'].interpolate(method='quadratic', limit_direction='both')
data['tp'] = data['tp'].clip(lower=0)
#check the nan  values
nan_values = data['tp'].isna().any()
nan_percentage = (data['tp'].isna().sum() / len(data['tp'])) * 100
print((data))

       longitude  latitude                 time        tp  expver
0          136.5      -7.5  2022-01-01 00:00:00  0.000095     NaN
1          136.5      -7.5  2022-01-01 01:00:00  0.000165     NaN
2          136.5      -7.5  2022-01-01 02:00:00  0.000260     NaN
3          136.5      -7.5  2022-01-01 03:00:00  0.000374     NaN
4          136.5      -7.5  2022-01-01 04:00:00  0.000357     NaN
...          ...       ...                  ...       ...     ...
14515      136.5      -7.5  2023-04-30 21:00:00  0.001546     5.0
14516      136.5      -7.5  2023-04-30 22:00:00  0.001260     1.0
14517      136.5      -7.5  2023-04-30 22:00:00  0.000890     5.0
14518      136.5      -7.5  2023-04-30 23:00:00  0.000609     1.0
14519      136.5      -7.5  2023-04-30 23:00:00  0.000415     5.0

[14520 rows x 5 columns]


### Check if there are any nan values

In [ ]:
# Print the result
print("Percentage of NaN values in the Precipitation data: {:.2f}%".format(nan_percentage))
# Print the result
if nan_values:
    print("There are NaN values in the Precipitation data.")
else:
    print("No NaN values found in the Precipitation data.")
print(data)
# daerah = data

Percentage of NaN values in the Precipitation data: 0.00%
No NaN values found in the Precipitation data.
       longitude  latitude                 time        tp  expver
0          136.5      -7.5  2022-01-01 00:00:00  0.000095     NaN
1          136.5      -7.5  2022-01-01 01:00:00  0.000165     NaN
2          136.5      -7.5  2022-01-01 02:00:00  0.000260     NaN
3          136.5      -7.5  2022-01-01 03:00:00  0.000374     NaN
4          136.5      -7.5  2022-01-01 04:00:00  0.000357     NaN
...          ...       ...                  ...       ...     ...
14515      136.5      -7.5  2023-04-30 21:00:00  0.001546     5.0
14516      136.5      -7.5  2023-04-30 22:00:00  0.001260     1.0
14517      136.5      -7.5  2023-04-30 22:00:00  0.000890     5.0
14518      136.5      -7.5  2023-04-30 23:00:00  0.000609     1.0
14519      136.5      -7.5  2023-04-30 23:00:00  0.000415     5.0

[14520 rows x 5 columns]


In [ ]:
X_train, y_train, X_val, y_val, train, val, test,j = splitting_data(data)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

[[-0.34128057]
 [-0.22120583]
 [-0.0580873 ]
 ...
 [ 1.0278211 ]
 [ 0.54278195]
 [ 0.20880856]]


((11605, 9, 1), (11605, 3), (4, 9, 1), (4, 3))

In [ ]:
train.shape, val.shape, test.shape

((3872, 3, 1), (5, 3, 1), (484, 3, 1))

In [ ]:
# fitting model to data 
model = define_model()
history = fit_model(X_train, y_train, X_val, y_val)

Epoch 1/10
363/363 - 17s - loss: 0.6695 - accuracy: 0.3702 - val_loss: 0.1925 - val_accuracy: 0.7500 - 17s/epoch - 47ms/step
Epoch 2/10
363/363 - 8s - loss: 0.5319 - accuracy: 0.4162 - val_loss: 0.2236 - val_accuracy: 0.5000 - 8s/epoch - 22ms/step
Epoch 3/10
363/363 - 8s - loss: 0.5110 - accuracy: 0.4201 - val_loss: 0.2413 - val_accuracy: 0.2500 - 8s/epoch - 21ms/step
Epoch 4/10
363/363 - 8s - loss: 0.5015 - accuracy: 0.4181 - val_loss: 0.2148 - val_accuracy: 0.5000 - 8s/epoch - 23ms/step
Epoch 5/10
363/363 - 7s - loss: 0.4962 - accuracy: 0.4202 - val_loss: 0.1126 - val_accuracy: 0.5000 - 7s/epoch - 19ms/step
Epoch 6/10
363/363 - 9s - loss: 0.4896 - accuracy: 0.4216 - val_loss: 0.1340 - val_accuracy: 0.2500 - 9s/epoch - 24ms/step
Epoch 7/10
363/363 - 7s - loss: 0.4861 - accuracy: 0.4233 - val_loss: 0.1735 - val_accuracy: 0.2500 - 7s/epoch - 19ms/step
Epoch 8/10
363/363 - 8s - loss: 0.4785 - accuracy: 0.4118 - val_loss: 0.1720 - val_accuracy: 0.2500 - 8s/epoch - 23ms/step
Epoch 9/10
363

In [ ]:
test_results = forecast(data,'WPP718')
test_results

,Predictions,Actuals,Date
0,0.000497,0.001126,2023-03-31 18:00:00
1,0.000537,0.001438,2023-03-31 18:00:00
2,0.000500,0.002393,2023-03-31 19:00:00
3,0.002247,0.002575,2023-03-31 19:00:00
4,0.001546,0.001945,2023-03-31 20:00:00
...,...,...,...
1447,0.001037,0.001546,2023-04-30 21:00:00
1448,0.000897,0.001260,2023-04-30 22:00:00
1449,0.000842,0.000890,2023-04-30 22:00:00
1450,0.000803,0.000609,2023-04-30 23:00:00


In [ ]:
forecast_viz() 


In [ ]:
error= evaluate()
error

,values
rmse,0.00034
corr,0.82274
bias,-0.00005
pvalue,0.00923


In [ ]:
saved_model_path = '/content/drive/MyDrive/overfishnt/data/weather_and_marine_condition/presipitasi/csv'+'presipitasiwpp718'+'.h5'
model.save(saved_model_path)